# Customer Churn Prediction & Revenue at Risk Analysis

**Course:** Quantitative Risk Management — Final Project  
**Author:** Eduard Aguado Marin  

## Objective

This notebook builds a machine learning pipeline to predict customer churn in a telecom company and quantify the **Revenue at Risk (RAR)** — the annual revenue exposed to loss from customers likely to leave.

**Pipeline overview:**
1. **Data Cleaning & Feature Engineering** — parse European-format monetary columns, bucketize tenure, encode categoricals
2. **Exploratory Data Analysis** — distributions, correlations, PCA dimensionality reduction, K-Means clustering
3. **Modelling** — Logistic Regression (baseline), Random Forest, XGBoost with GridSearchCV; SMOTE oversampling to handle class imbalance
4. **Model Interpretation** — SHAP values, permutation importance, ROC curves
5. **Revenue at Risk** — translate churn probabilities into dollar-denominated business risk

In [ ]:
# ── Imports ──────────────────────────────────────────────────────────────────
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shap
import xgboost as xgb

from collections import Counter
from imblearn.over_sampling import SMOTE

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import (
    silhouette_score, confusion_matrix, classification_report,
    roc_auc_score, roc_curve
)
from sklearn.model_selection import (
    train_test_split, GridSearchCV, StratifiedKFold
)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance

# Plotting defaults
sns.set_style('whitegrid')
plt.rcParams['figure.dpi'] = 100

---
# 1. Data Loading & Initial Inspection

The dataset contains **7,043 telecom customer records** with 23 columns covering demographics, service subscriptions, billing, support tickets, and the binary churn label.

> **Note:** The CSV uses a semicolon separator and European decimal format (commas instead of dots), so `MonthlyCharges` and `TotalCharges` are initially read as strings.

In [ ]:
df = pd.read_csv('telcom_dataset.csv', sep=';')
df.replace('', np.nan, inplace=True)

print(f"Shape: {df.shape}")
print(f"\nMissing values:\n{df.isna().sum()[df.isna().sum() > 0]}"
      if df.isna().any().any() else "No missing values found.")
df.head()

In [ ]:
# Unique values per column — helps decide encoding strategy
# Binary (2 unique) → 0/1, Few categories (3-4) → one-hot, Continuous → keep as-is
df.nunique()

In [ ]:
# MonthlyCharges and TotalCharges are stored as object (string) due to European comma decimals
df.dtypes

In [ ]:
# Target variable distribution — 26.5% churn rate indicates moderate class imbalance
df.Churn.value_counts(normalize=True)

---
# 2. Data Cleaning & Feature Engineering

Steps:
1. Drop `customerID` (not predictive)
2. Parse `MonthlyCharges` / `TotalCharges` from European string format → float
3. Bucketize `tenure` into 7 interpretable groups (0-6 mo, 6-12 mo, ..., 60+ mo)
4. Encode binary columns (Yes/No → 1/0) and one-hot encode multi-category columns

In [ ]:
# Drop customerID — it's a unique identifier, not a feature
if 'customerID' in df.columns:
    df.drop('customerID', axis=1, inplace=True)

def clean_monetary_columns(df, cols=('TotalCharges', 'MonthlyCharges')):
    """Convert European-format monetary strings to float.
    
    Handles: '1.234,56' (dot thousands, comma decimal), '12,5' (comma decimal only),
    empty strings, and NaN values.
    """
    for col in cols:
        if col not in df.columns:
            continue
        s = df[col].astype(str).str.strip()
        s = s.mask((s == '') | (s.str.lower() == 'nan'), '0')

        has_comma = s.str.contains(',', regex=False)
        has_dot = s.str.contains('.', regex=False)
        s2 = s.copy()
        mask_both = has_comma & has_dot

        # '1.234,56' → remove dot thousands separator, then comma → dot
        if mask_both.any():
            s2.loc[mask_both] = (
                s.loc[mask_both]
                .str.replace('.', '', regex=False)
                .str.replace(',', '.', regex=False)
            )

        # '12,5' → '12.5'
        only_comma = has_comma & ~mask_both
        if only_comma.any():
            s2.loc[only_comma] = s.loc[only_comma].str.replace(',', '.', regex=False)

        s2 = s2.str.replace(r'[^0-9\.\-]', '', regex=True)
        df[col] = pd.to_numeric(s2, errors='coerce').fillna(0.0)

    return df

df = clean_monetary_columns(df)
print(f"MonthlyCharges dtype: {df['MonthlyCharges'].dtype}, range: [{df['MonthlyCharges'].min():.2f}, {df['MonthlyCharges'].max():.2f}]")
print(f"TotalCharges   dtype: {df['TotalCharges'].dtype},   range: [{df['TotalCharges'].min():.2f}, {df['TotalCharges'].max():.2f}]")

In [ ]:
# Bucketize tenure into interpretable groups
# Raw tenure has 73 unique values (months) — grouping reduces noise and aids interpretation
TENURE_LABELS = {1: '0–6 mo', 2: '6–12 mo', 3: '12–24 mo',
                 4: '24–36 mo', 5: '36–48 mo', 6: '48–60 mo', 7: '60+ mo'}

def bucketize_tenure(t):
    if t <= 6:   return 1
    if t <= 12:  return 2
    if t <= 24:  return 3
    if t <= 36:  return 4
    if t <= 48:  return 5
    if t <= 60:  return 6
    return 7

df['tenure_group'] = df['tenure'].apply(bucketize_tenure)
df.drop('tenure', axis=1, inplace=True)

print("Tenure group distribution:")
print(df['tenure_group'].value_counts().sort_index().rename(TENURE_LABELS))

### 2.1 Feature Encoding

Encoding strategy:
- **Binary Yes/No columns** → 0/1 (including those with "No internet service" / "No phone service" mapped to 0)
- **Gender** → binary `male` column (Female=0, Male=1)
- **Multi-category columns** (`InternetService`, `Contract`, `PaymentMethod`) → one-hot encoding with `drop_first=True` to avoid multicollinearity

In [ ]:
# Gender → binary
df['male'] = df['gender'].map({'Female': 0, 'Male': 1})
df.drop('gender', axis=1, inplace=True)

# Yes/No columns → 0/1
for col in ['Partner', 'Dependents', 'PaperlessBilling', 'Churn', 'PhoneService']:
    df[col] = df[col].map({'No': 0, 'Yes': 1})

# Columns with "No internet service" → treat as 0 (service not used)
for col in ['OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
            'TechSupport', 'StreamingTV', 'StreamingMovies']:
    df[col] = df[col].map({'No internet service': 0, 'No': 0, 'Yes': 1})

# MultipleLines: "No phone service" → 0
df['MultipleLines'] = df['MultipleLines'].map({'No phone service': 0, 'No': 0, 'Yes': 1})

# One-hot encode multi-category features (drop_first avoids perfect multicollinearity)
df = pd.get_dummies(df, columns=['InternetService', 'Contract', 'PaymentMethod'], drop_first=True)
df = df.astype({col: int for col in df.select_dtypes('bool').columns})

# Sanitize column names (replace spaces with underscores)
df.columns = df.columns.str.replace(' ', '_')

print(f"Final shape: {df.shape}")
df.head()

In [ ]:
# Final class distribution after encoding
churn_counts = df['Churn'].value_counts()
print(f"No Churn: {churn_counts[0]} ({churn_counts[0]/len(df)*100:.1f}%)")
print(f"Churn:    {churn_counts[1]} ({churn_counts[1]/len(df)*100:.1f}%)")
print(f"\nTotal features: {df.shape[1] - 1} + 1 target = {df.shape[1]} columns")

---
# 3. Exploratory Data Analysis

We examine the data from three angles:
1. **Univariate distributions** — understand the shape of each feature
2. **Feature correlations** — identify multicollinearity and features associated with churn
3. **PCA + K-Means clustering** — discover natural customer segments and their churn concentration

### 3.1 Target Variable Distribution

In [ ]:
# ── Helper: plot distributions for a set of columns ──
def plot_variable_distributions(df, title_prefix=''):
    num_vars = df.select_dtypes(include=['int64', 'float64']).columns
    cat_vars = df.select_dtypes(include=['object', 'category']).columns
    total_vars = len(num_vars) + len(cat_vars)
    cols = 3
    rows = (total_vars + cols - 1) // cols
    fig, axes = plt.subplots(rows, cols, figsize=(cols * 5, rows * 4))
    axes = axes.flatten()

    for i, col in enumerate(num_vars):
        if set(df[col].dropna().unique()).issubset({0, 1}):
            sns.countplot(x=df[col], ax=axes[i])
            axes[i].set_xticks([0, 1])
        else:
            sns.histplot(df[col], bins=20, kde=False, ax=axes[i])
        axes[i].set_title(f'{title_prefix}{col}')

    for j, col in enumerate(cat_vars, start=len(num_vars)):
        sns.countplot(x=df[col], ax=axes[j])
        axes[j].set_title(f'{title_prefix}{col}')
        axes[j].tick_params(axis='x', rotation=45)

    for k in range(total_vars, len(axes)):
        fig.delaxes(axes[k])

    plt.tight_layout()
    plt.show()

In [ ]:
# Churn vs Non-Churn bar plot with counts and percentages
churn_counts = df['Churn'].value_counts()
churn_percent = df['Churn'].value_counts(normalize=True) * 100

fig, ax = plt.subplots(figsize=(8, 6))
sns.barplot(x=churn_counts.index, y=churn_counts.values, palette='viridis', ax=ax)
for i, (count, pct) in enumerate(zip(churn_counts.values, churn_percent.values)):
    ax.text(i, count + 50, f'{count} ({pct:.1f}%)', ha='center', va='bottom', fontsize=12)
ax.set_xticklabels(['No Churn', 'Churn'])
ax.set_ylabel('Number of Customers')
ax.set_title('Customer Churn Distribution')
plt.show()

### 3.2 Feature Distributions

We split features into **binary** (0/1) and **non-binary** groups for cleaner visualization.

In [ ]:
# Binary features (0/1)
binary_vars = [col for col in df.columns if set(df[col].dropna().unique()).issubset({0, 1})]
plot_variable_distributions(df[binary_vars])

In [ ]:
# Non-binary features (continuous and multi-category)
other_vars = [col for col in df.columns if col not in binary_vars]
plot_variable_distributions(df[other_vars])

### 3.3 Feature Correlations

The heatmap below highlights which features are most correlated with `Churn` and with each other. Strong inter-feature correlations (e.g., `MonthlyCharges` ↔ streaming services) may affect logistic regression coefficients but are handled well by tree-based models.

In [ ]:
plt.figure(figsize=(14, 11))
sns.heatmap(df.corr(numeric_only=True), annot=True, fmt='.1f', cmap='coolwarm', square=True,
            linewidths=0.5, cbar_kws={'shrink': 0.8})
plt.title('Feature Correlation Heatmap')
plt.tight_layout()
plt.show()

### 3.4 PCA & K-Means Clustering

**Goal:** Identify natural customer segments and check whether churn concentrates in specific clusters.

**Approach:**
- Standardize features, then apply PCA retaining 95% of variance
- Use the **elbow method** (WCSS) and **silhouette scores** to evaluate cluster quality
- Visualize clusters in the first two principal components, split by churn status

> **Note on k selection:** The silhouette score is maximized at k=2, but we choose **k=3** because it produces more interpretable segments — the 3-cluster solution clearly separates a high-churn group (Cluster 0) from two lower-churn groups, whereas k=2 merges these distinct behavioral profiles into a single heterogeneous cluster.

In [ ]:
# ── PCA ──────────────────────────────────────────────────────────────────────
X = df.drop(columns='Churn')
y = df['Churn']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

pca_full = PCA(n_components=0.95)
X_pca = pca_full.fit_transform(X_scaled)
print(f"PCA retained {X_pca.shape[1]} components, explaining "
      f"{pca_full.explained_variance_ratio_.sum():.1%} of variance")

# ── Evaluate k with silhouette scores ────────────────────────────────────────
k_range = range(2, 11)
sil_scores = []
for k in k_range:
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = km.fit_predict(X_pca)
    sil_scores.append(silhouette_score(X_pca, labels))

print("\nSilhouette scores:", dict(zip(k_range, [round(s, 3) for s in sil_scores])))
print(f"Best k by silhouette: {list(k_range)[sil_scores.index(max(sil_scores))]}")

# We override the silhouette-optimal k with k=3 for better interpretability
# (see markdown note above — k=3 separates the high-churn group more clearly)
CHOSEN_K = 3
print(f"Selected k = {CHOSEN_K}")

# ── Elbow method (WCSS) ─────────────────────────────────────────────────────
k_values = range(1, 16)
wcss = []
for k in k_values:
    km = KMeans(n_clusters=k, random_state=42, n_init=20)
    km.fit(X_pca)
    wcss.append(km.inertia_)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(list(k_values), wcss, marker='o', linewidth=2)
axes[0].axvline(CHOSEN_K, color='red', linestyle='--', alpha=0.7, label=f'Chosen k={CHOSEN_K}')
axes[0].set_xlabel('Number of clusters k')
axes[0].set_ylabel('WCSS (inertia)')
axes[0].set_title('Elbow Method')
axes[0].set_xticks(list(k_values))
axes[0].legend()
axes[0].grid(alpha=0.3)

axes[1].plot(list(k_range), sil_scores, marker='s', linewidth=2, color='orange')
axes[1].axvline(CHOSEN_K, color='red', linestyle='--', alpha=0.7, label=f'Chosen k={CHOSEN_K}')
axes[1].set_xlabel('Number of clusters k')
axes[1].set_ylabel('Silhouette Score')
axes[1].set_title('Silhouette Analysis')
axes[1].set_xticks(list(k_range))
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# ── Fit K-Means with k=3 and visualize ───────────────────────────────────────
kmeans = KMeans(n_clusters=CHOSEN_K, random_state=42, n_init=20)
clusters = kmeans.fit_predict(X_pca)

X_plot = X_pca[:, :2]
plot_df = pd.DataFrame(X_plot, columns=['PC1', 'PC2']).assign(cluster=clusters, churn=y.values)

# Side-by-side scatter plots: Non-Churned vs Churned
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

sns.scatterplot(data=plot_df[plot_df['churn'] == 0], x='PC1', y='PC2', hue='cluster',
                s=50, alpha=0.8, edgecolor='k', linewidth=0.3, ax=axes[0])
axes[0].set_title('Non-Churned Customers')

sns.scatterplot(data=plot_df[plot_df['churn'] == 1], x='PC1', y='PC2', hue='cluster',
                s=50, alpha=0.8, edgecolor='k', linewidth=0.3, ax=axes[1])
axes[1].set_title('Churned Customers')

plt.tight_layout()
plt.show()

# Churn distribution per cluster
print("\nCluster sizes and churn rates:")
ct = pd.crosstab(plot_df['cluster'], plot_df['churn'], margins=True)
ct.columns = ['No Churn', 'Churn', 'Total']
ct['Churn Rate'] = (ct['Churn'] / ct['Total'] * 100).round(1).astype(str) + '%'
print(ct)

In [ ]:
# PCA loadings — which original features contribute most to PC1 and PC2
pc_df = pd.DataFrame(
    pca_full.components_[:2].T,
    index=X.columns,
    columns=['PC1', 'PC2']
).sort_values('PC1', ascending=False)

print("Top PC1 drivers: MonthlyCharges, TotalCharges, StreamingTV/Movies (service engagement)")
print("Top PC2 drivers: Contract_Two_year, tenure_group (customer loyalty) vs Electronic check, Fiber optic (churn risk)")
pc_df

**Key finding:** ~75% of all churned customers are concentrated in **Cluster 0** (1,272 out of 1,869). This cluster has a ~49.5% churn rate, compared to ~15-18% in the other clusters. Feature importance analysis in the modelling section will help explain *why* these customers churn — likely driven by short tenure, fiber optic internet, month-to-month contracts, and high tech ticket counts.

---
# 4. Modelling

We train three classifiers of increasing complexity:

| Model | Purpose |
|-------|---------|
| **Logistic Regression** | Interpretable linear baseline |
| **Random Forest** | Non-linear ensemble with built-in feature importance |
| **XGBoost** | Gradient-boosted trees, hyperparameter-tuned via GridSearchCV |

**Class imbalance handling:**
- **SMOTE** (Synthetic Minority Over-sampling Technique) is applied to the **training set only** to avoid data leakage into the test set
- Random Forest also uses `class_weight={0: 1, 1: 2}` as additional imbalance correction

### 4.1 Train/Test Split & Helper Functions

In [ ]:
# ── Train/test split (80/20, stratified on Churn) ────────────────────────────
feature_cols = df.columns[df.columns != 'Churn']
train, test = train_test_split(df, test_size=0.2, random_state=42, stratify=df['Churn'])

X_train = train[feature_cols]
Y_train = train['Churn']
X_test  = test[feature_cols]
Y_test  = test['Churn']

print(f"Train: {X_train.shape[0]} samples | Test: {X_test.shape[0]} samples")
print(f"Train churn rate: {Y_train.mean():.1%} | Test churn rate: {Y_test.mean():.1%}")

# ── Helper functions ─────────────────────────────────────────────────────────
def print_metrics(model, X_train, Y_train, X_test, Y_test, label="Model"):
    """Print classification reports for train and test sets."""
    print(f"\n{'='*60}")
    print(f"  {label} — Classification Report")
    print(f"{'='*60}")
    print("\nTraining Set:")
    print(classification_report(Y_train, model.predict(X_train)))
    print("Test Set:")
    print(classification_report(Y_test, model.predict(X_test)))

def plot_confusion_matrix(y_true, y_pred, title="Confusion Matrix"):
    """Plot a heatmap confusion matrix."""
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(6, 4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
                xticklabels=['No Churn', 'Churn'], yticklabels=['No Churn', 'Churn'])
    plt.title(title)
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.tight_layout()
    plt.show()

### 4.2 SMOTE Oversampling

SMOTE generates synthetic minority-class samples by interpolating between existing minority neighbors. It is applied **only to the training set** — the test set retains the original class distribution to give realistic performance estimates.

In [ ]:
print("Before SMOTE:", Counter(Y_train))
smote = SMOTE(random_state=42)
X_train, Y_train = smote.fit_resample(X_train, Y_train)
print("After  SMOTE:", Counter(Y_train))

In [ ]:
# Visualize class balance after SMOTE (training) vs original (test)
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for ax, data, title in [(axes[0], Y_train, 'Training Set (After SMOTE)'),
                         (axes[1], Y_test, 'Test Set (Original Distribution)')]:
    counts = data.value_counts().sort_index()
    pcts = (counts / counts.sum() * 100)
    sns.barplot(x=counts.index, y=counts.values, palette='viridis', ax=ax)
    for i, (cnt, pct) in enumerate(zip(counts.values, pcts.values)):
        ax.text(i, cnt + cnt * 0.02, f'{cnt} ({pct:.1f}%)', ha='center', fontsize=11)
    ax.set_xticks([0, 1])
    ax.set_xticklabels(['No Churn', 'Churn'])
    ax.set_ylabel('Count')
    ax.set_title(title)

plt.tight_layout()
plt.show()

### 4.3 Model 1: Logistic Regression (Baseline)

A simple linear model that serves as our baseline. Since SMOTE already balances the training data, we use default settings with `max_iter=1000` to ensure convergence.

In [ ]:
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, Y_train)

print_metrics(lr, X_train, Y_train, X_test, Y_test, label="Logistic Regression")
plot_confusion_matrix(Y_test, lr.predict(X_test), title="Logistic Regression — Confusion Matrix (Test Set)")

### 4.4 Model 2: Random Forest

Key hyperparameters:
- `n_estimators=1000` — large ensemble for stable predictions
- `criterion='entropy'` — information gain split criterion
- `max_depth=10` — limits tree depth to reduce overfitting
- `class_weight={0: 1, 1: 2}` — additional penalty for misclassifying churned customers (on top of SMOTE)

In [ ]:
rf = RandomForestClassifier(
    n_estimators=1000, criterion='entropy', max_depth=10,
    max_features='sqrt', min_samples_split=8,
    class_weight={0: 1, 1: 2}, random_state=42, n_jobs=-1
)
rf.fit(X_train, Y_train)

print_metrics(rf, X_train, Y_train, X_test, Y_test, label="Random Forest")
plot_confusion_matrix(Y_test, rf.predict(X_test), title="Random Forest — Confusion Matrix (Test Set)")

# Gini-based feature importance
importances = pd.Series(rf.feature_importances_, index=X_train.columns).sort_values(ascending=False)
print("\nTop 10 features (Gini importance):")
print(importances.head(10).to_string())

In [ ]:
# Permutation importance — more reliable than Gini for correlated features
# Measures actual accuracy drop when each feature is randomly shuffled
perm_result = permutation_importance(rf, X_test, Y_test, n_repeats=30, random_state=42, n_jobs=-1)
perm_importances = pd.Series(perm_result.importances_mean, index=X_test.columns).sort_values(ascending=False)

print("Top 10 features (permutation importance):")
print(perm_importances.head(10).to_string())

### 4.5 Model 3: XGBoost (with GridSearchCV)

XGBoost is tuned via exhaustive grid search over key hyperparameters, using **5-fold stratified cross-validation** and **ROC-AUC** as the scoring metric (better suited for imbalanced targets than accuracy).

In [ ]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

param_grid_xgb = {
    'n_estimators':      [200, 500, 1000],
    'max_depth':         [3, 6, 10],
    'learning_rate':     [0.01, 0.1, 0.2],
    'subsample':         [0.7, 1.0],
    'colsample_bytree':  [0.7, 1.0],
    'scale_pos_weight':  [1, 2],
}

gs_xgb = GridSearchCV(
    xgb.XGBClassifier(random_state=42, n_jobs=-1, eval_metric='logloss'),
    param_grid_xgb,
    scoring='roc_auc',
    cv=cv,
    n_jobs=-1,
    verbose=1
)
gs_xgb.fit(X_train, Y_train)

print(f"\nBest CV ROC-AUC: {gs_xgb.best_score_:.4f}")
print(f"Best parameters: {gs_xgb.best_params_}")

xgb_model = gs_xgb.best_estimator_
print_metrics(xgb_model, X_train, Y_train, X_test, Y_test, label="XGBoost")
plot_confusion_matrix(Y_test, xgb_model.predict(X_test), title="XGBoost — Confusion Matrix (Test Set)")

### 4.6 Model Comparison — ROC Curves

The ROC curve plots the True Positive Rate vs False Positive Rate at various classification thresholds. A higher AUC (Area Under Curve) indicates better discrimination between churned and non-churned customers.

In [ ]:
models = {
    "Logistic Regression": lr,
    "Random Forest": rf,
    "XGBoost": xgb_model
}

fig, axes = plt.subplots(1, 3, figsize=(18, 5))
for ax, (label, model) in zip(axes, models.items()):
    y_proba = model.predict_proba(X_test)[:, 1]
    fpr, tpr, _ = roc_curve(Y_test, y_proba)
    auc = roc_auc_score(Y_test, y_proba)
    ax.plot(fpr, tpr, linewidth=2, label=f'AUC = {auc:.3f}')
    ax.plot([0, 1], [0, 1], 'k--', alpha=0.5)
    ax.set_title(f'ROC Curve — {label}')
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    ax.legend(loc='lower right')
    ax.grid(alpha=0.3)
plt.tight_layout()
plt.show()

### 4.7 SHAP Values (Model Interpretability)

SHAP (SHapley Additive exPlanations) assigns each feature a contribution to each individual prediction:
- **Bar plot** — global feature importance (mean |SHAP| across all test samples)
- **Beeswarm plot** — shows both importance *and* direction of effect (red = high feature value, blue = low)

In [ ]:
# Compute SHAP values for the Random Forest model
explainer = shap.Explainer(rf)
shap_values = explainer(X_test)

# Global feature importance
shap.plots.bar(shap_values, max_display=15, show=False)
plt.title('SHAP — Global Feature Importance (Random Forest)')
plt.tight_layout()
plt.show()

# Beeswarm: direction + magnitude of feature effects
shap.plots.beeswarm(shap_values, max_display=10, show=False)
plt.title('SHAP — Beeswarm Plot (Random Forest)')
plt.tight_layout()
plt.show()

---
# 5. Revenue at Risk (RAR) Analysis

The final step translates model predictions into **business value**. We use the Random Forest model to estimate each customer's churn probability, then calculate:

- **Revenue at Risk** = sum of annualized charges for *currently non-churned* customers whose predicted churn probability exceeds 50%
- This represents the revenue that the company stands to lose if no retention action is taken

> We exclude already-churned customers from the RAR calculation since their revenue is already lost.

### 5.1 Churn Probability Distribution

In [ ]:
# Predict churn probability for all customers using Random Forest
df['Churn_Probability'] = rf.predict_proba(df[feature_cols])[:, 1]

# Sanity check: mean probability should be high for actual churners, low for non-churners
means = df.groupby('Churn')['Churn_Probability'].mean()
print("Mean churn probability by actual status:")
print(f"  Non-churned (0): {means[0]:.3f}")
print(f"  Churned     (1): {means[1]:.3f}")

In [ ]:
# Probability distribution by actual churn status
fig, ax = plt.subplots(figsize=(9, 6))
sns.histplot(data=df, x='Churn_Probability', hue='Churn', bins=30, kde=True,
             stat='density', common_norm=False, palette={0: 'steelblue', 1: 'coral'}, alpha=0.6, ax=ax)

# Mark the means
ymax = ax.get_ylim()[1]
for cls, color, label in [(0, 'steelblue', 'No Churn'), (1, 'coral', 'Churn')]:
    m = means[cls]
    ax.axvline(m, color=color, linestyle='--', linewidth=2)
    ax.text(m, ymax * 0.92, f'{label} mean: {m:.3f}', color='white',
            bbox=dict(facecolor=color, edgecolor='none', alpha=0.85), ha='center', fontsize=10)

ax.set_title('Churn Probability Distribution by Actual Churn Status')
ax.set_xlabel('Predicted Churn Probability')
plt.tight_layout()
plt.show()

### 5.2 Revenue at Risk Calculation

In [ ]:
# Annualize monthly charges
df['AnnualCharges'] = df['MonthlyCharges'] * 12

# Filter: non-churned customers with predicted churn probability > 50%
at_risk = df[(df['Churn_Probability'] > 0.5) & (df['Churn'] == 0)]
total_non_churned_revenue = df[df['Churn'] == 0]['AnnualCharges'].sum()

revenue_at_risk = at_risk['AnnualCharges'].sum()
rar_pct = revenue_at_risk / total_non_churned_revenue * 100

print(f"{'='*55}")
print(f"  REVENUE AT RISK SUMMARY")
print(f"{'='*55}")
print(f"  At-risk customers:        {len(at_risk):,}")
print(f"  Revenue at Risk:           ${revenue_at_risk:,.2f} / year")
print(f"  Total non-churned revenue: ${total_non_churned_revenue:,.2f} / year")
print(f"  RAR as % of revenue:       {rar_pct:.2f}%")
print(f"{'='*55}")

---
# 6. Conclusions

## Key Findings

1. **Class imbalance:** Only 26.5% of customers churned. SMOTE oversampling and class weighting were used to prevent models from defaulting to the majority class.

2. **Customer segmentation:** K-Means clustering (k=3) revealed that **Cluster 0 concentrates ~75% of all churned customers** with a ~49.5% churn rate, compared to ~15-18% in the other clusters.

3. **Top churn drivers** (by permutation importance and SHAP):
   - **Number of tech support tickets** — by far the strongest predictor
   - **No internet service** — customers without internet churn less
   - **Two-year contracts** — strong protective effect against churn
   - **Monthly charges** — higher charges correlate with higher churn risk

4. **Revenue at Risk:** The Random Forest model identifies **969 currently non-churned customers** with >50% predicted churn probability, representing **$743,722 in annual revenue at risk** (19.55% of total non-churned revenue).

## Business Implications

- **Proactive retention:** Target the 969 at-risk customers with retention offers (discounts, contract upgrades, loyalty programs)
- **Improve tech support:** The number of tech tickets is the dominant churn signal — improving service quality and first-contact resolution could directly reduce churn
- **Promote long-term contracts:** Two-year contracts significantly reduce churn risk; incentivize month-to-month customers to switch